# Análise de Preços de Imóveis na Califórnia

Este notebook demonstra como analisar um conjunto de dados de preços de imóveis da Califórnia para criar um modelo de previsão de preços. O objetivo é entender os fatores que influenciam os preços dos imóveis e preparar os dados para machine learning.

## Carregamento dos Dados

A função `load_housing_data()` realiza as seguintes operações:
1. Procura pelo arquivo **datasets/housing.tgz** no diretório atual
2. Se não encontrar:
   - Cria o diretório **datasets**
   - Baixa o arquivo do repositório GitHub **ageron/data**
   - Extrai o conteúdo criando **datasets/housing/housing.csv**
3. Carrega e retorna os dados em um DataFrame do Pandas

> **Nota**: O arquivo CSV contém informações sobre distritos censitários da Califórnia, onde cada linha representa um distrito com suas características demográficas e imobiliárias.

In [ ]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import matplotlib.pyplot as plt

def load_housing_data():
    csv_path = Path("datasets/housing/housing.csv")
    if csv_path.is_file():
        return pd.read_csv(csv_path)
        
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
    
    with tarfile.open(tarball_path) as housing_tarball:
        housing_tarball.extractall(path="datasets")
    return pd.read_csv(csv_path)


housing = load_housing_data()

## Exploração Inicial dos Dados

Vamos começar examinando a estrutura dos dados. Cada linha do DataFrame representa um distrito censitário da Califórnia, com os seguintes atributos:

- **longitude, latitude**: Localização geográfica do distrito
- **housing_median_age**: Idade mediana das casas no distrito
- **total_rooms**: Número total de cômodos
- **total_bedrooms**: Número total de quartos
- **population**: População total do distrito
- **households**: Número de domicílios
- **median_income**: Renda mediana da região (em dezenas de milhares de dólares)
- **median_house_value**: Valor mediano das casas (nosso atributo alvo)
- **ocean_proximity**: Categoria indicando proximidade com o oceano

> **Nota**: O atributo `ocean_proximity` não aparece nos histogramas por ser categórico (não-numérico). Analisaremos sua distribuição separadamente usando `value_counts()`.

Primeiro, vamos observar as cinco primeiras linhas usando o método `head()` do DataFrame:

In [ ]:
housing.head()

Cada linha representa um distrito. Existem **10 atributos** (nem todos são mostrados na captura de tela): `longitude`, `latitude`, `housing_median_age`, `total_rooms`, `total_bedrooms`, `population`, `households`, `median_income`, `median_house_value` e `ocean_proximity`.

***

O método `info()` é útil para obter uma descrição rápida dos dados, em particular o número total de linhas, o tipo de cada atributo e o número de valores não nulos:

In [ ]:
housing.info()

## Análise da Estrutura dos Dados

A saída do método `info()` nos revela informações importantes:

1. **Tamanho do Dataset**: 
   - 20.640 instâncias (distritos)
   - 10 atributos (características)

2. **Tipos de Dados**:
   - 9 atributos numéricos (tipo `float64`)
   - 1 atributo categórico (`ocean_proximity`)

3. **Valores Ausentes**:
   - `total_bedrooms`: 207 valores ausentes (1% dos dados)
   - Todos os outros atributos estão completos

Esta análise inicial já nos indica que precisaremos:
- Tratar os valores ausentes em `total_bedrooms`
- Converter o atributo categórico para formato numérico
- Considerar normalização dos dados devido às diferentes escalas

In [ ]:
housing["ocean_proximity"].value_counts()

In [ ]:
housing.describe()

As linhas `count` (contagem), `mean` (média), `min` (mínimo) e `max` (máximo) são autoexplicativas. Note que os valores nulos são ignorados (assim, por exemplo, a contagem de `total_bedrooms` é 20.433, e não 20.640). A linha `std` mostra o desvio padrão (*standard deviation*), que mede quão dispersos os valores estão.

As linhas `25%`, `50%` e `75%` mostram os percentis correspondentes: um percentil indica o valor abaixo do qual uma determinada porcentagem de observações em um grupo se encontra. Por exemplo, 25% dos distritos têm um `housing_median_age` inferior a 18, enquanto 50% estão abaixo de 29 e 75% estão abaixo de 37. Estes são frequentemente chamados de **25º percentil** (ou **primeiro quartil**), a **mediana** e o **75º percentil** (ou **terceiro quartil**).

***

Outra maneira rápida de ter uma noção do tipo de dados com que você está lidando é plotar um histograma para cada atributo numérico. Um histograma mostra o número de instâncias (no eixo vertical) que possuem uma determinada faixa de valores (no eixo horizontal). Você pode plotar um atributo de cada vez ou pode chamar o método `hist()` em todo o conjunto de dados (como mostrado no exemplo de código a seguir), e ele plotará um histograma para cada atributo numérico:

In [ ]:
housing.hist(bins=50, figsize=(12, 8))
plt.show()

## Análise das Distribuições

A visualização dos histogramas revela características importantes dos nossos dados:

### 1. Atributos Pré-processados
- **Renda Mediana (`median_income`)**: 
  - Escalonada entre 0.5 e 15
  - Um valor de 3.0 representa $30,000 (valores divididos por 10,000)
  - Limitação artificial pode afetar a análise de áreas de alta renda

- **Valor Mediano do Imóvel (`median_house_value`)**:
  - Limitado em $500,000 (observe a linha vertical no histograma)
  - Esta limitação é um **problema crítico** pois:
    1. O modelo nunca aprenderá a prever valores acima deste limite
    2. Pode distorcer a relação real entre as variáveis
    3. Prejudica previsões em áreas de alto valor imobiliário

### 2. Distribuições Assimétricas
Vários atributos apresentam distribuição assimétrica à direita (cauda longa):
- `median_income`
- `total_rooms`
- `total_bedrooms`
- `population`
- `households`

Esta assimetria pode prejudicar o desempenho de alguns algoritmos de ML que assumem distribuição normal. Possíveis soluções:
- Transformação logarítmica
- Normalização dos dados
- Uso de algoritmos robustos a distribuições não-normais

### 3. Correlações Visuais
Os histogramas também sugerem possíveis correlações:
- `total_rooms` e `total_bedrooms` parecem ter distribuições similares
- `population` e `households` também mostram padrões semelhantes

## Próximos Passos Cruciais

### ⚠️ A Regra de Ouro do Machine Learning

Antes de qualquer pré-processamento ou análise adicional, devemos seguir a regra mais importante em machine learning:

**Separar o Conjunto de Teste (`Test Set`) ANTES de qualquer análise detalhada**

Por quê?
1. Evita "Data Leakage": Vazamento de informações do teste para o treino
2. Garante uma avaliação honesta do modelo
3. Previne overfitting por conhecimento prévio dos dados de teste

### Tarefas Pendentes

1. **Preparação dos Dados**:
   - Separar 20% dos dados para teste
   - Tratar os 207 valores ausentes em `total_bedrooms`
   - Aplicar transformações para normalizar distribuições
   - Realizar feature scaling nos atributos numéricos

2. **Tratamento de Limitações**:
   - Investigar impacto dos valores limitados
   - Considerar remoção ou marcação especial de casos limite
   - Documentar estas limitações para usuários do modelo

3. **Engenharia de Features**:
   - Criar novas features (ex: quartos por pessoa, cômodos por casa)
   - Codificar adequadamente `ocean_proximity`
   - Investigar interações entre variáveis